In [1]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.ollama import OllamaChatCompletionClient

# Create the agents.
model_client = OllamaChatCompletionClient(model="llama3.2")
assistant = AssistantAgent("assistant", model_client=model_client)
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("APPROVE")

# Create the team.
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a 4-line poem about the ocean.")
# Use asyncio.run(...) when running in a script.
await Console(stream)

---------- user ----------
Write a 4-line poem about the ocean.
---------- assistant ----------
Here is a 4-line poem about the ocean:

The ocean's waves crash on the shore,
A soothing sound that I adore.
Deep blue waters stretch far and wide,
A treasure trove, where secrets reside.
---------- user_proxy ----------

---------- assistant ----------
Would you like me to write another one?
---------- user_proxy ----------
more deeper
---------- assistant ----------
Here is a 4-line poem about the ocean with a more profound tone:

In the ocean's depths, I find my soul,
A world of darkness, yet a peaceful role.
The weight of waves upon my chest,
A reminder of life's eternal rest.
---------- user_proxy ----------
APPROVE


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a 4-line poem about the ocean.', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=59, completion_tokens=45), metadata={}, content="Here is a 4-line poem about the ocean:\n\nThe ocean's waves crash on the shore,\nA soothing sound that I adore.\nDeep blue waters stretch far and wide,\nA treasure trove, where secrets reside.", type='TextMessage'), UserInputRequestedEvent(source='user_proxy', models_usage=None, metadata={}, request_id='6c48840c-be4b-4fcb-a872-41cc8c0b807a', content='', type='UserInputRequestedEvent'), TextMessage(source='user_proxy', models_usage=None, metadata={}, content='', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=113, completion_tokens=10), metadata={}, content='Would you like me to write another one?', type='TextMessage'), UserInputRequestedEvent(source='user_proxy', models_usage=

In [ ]:
import time
from autogen.teamwork import GroupChat
from autogen import (
    OpenAIAgent,
    UserProxyAgent,
    # 필요하다면 더 import
)

# -------------------------------------------------------------------
# 1. 각 에이전트 정의
# -------------------------------------------------------------------

class UserAgent(UserProxyAgent):
    def __init__(self, name="UserAgent", keyword=None):
        super().__init__(name=name)
        self.keyword = keyword

    def set_keyword(self, keyword):
        self.keyword = keyword

    def act(self, message=None):
        if not self.keyword:
            raise ValueError("키워드가 설정되어 있지 않습니다.")
        # 실제 사용자 입력 받는 대신, 미리 세팅된 self.keyword를 직접 반환
        return self.keyword


class PoetAgent(OpenAIAgent):
    def __init__(self, name="PoetAgent", model="gpt-3.5-turbo"):
        super().__init__(name=name, model=model)
        # 이 에이전트가 기본적으로 참고할 system_prompt 지정
        self.system_prompt = (
            "당신은 창의적인 시인입니다.\n"
            "주어진 키워드를 바탕으로, 10줄짜리 아름다운 시를 작성하세요.\n"
            "항상 10줄을 정확히 지키세요.\n"
        )

class ComplimentAgent(OpenAIAgent):
    def __init__(self, name="ComplimentAgent", model="gpt-3.5-turbo"):
        super().__init__(name=name, model=model)
        self.system_prompt = (
            "당신은 시를 매우 긍정적으로 평가하는 평론가입니다.\n"
            "시를 보고 칭찬 위주의 코멘트를 남기되, 정말 좋다고 생각하면 반드시 '훌륭한 시야!'라고 말하세요."
        )

class CriticAgent(OpenAIAgent):
    def __init__(self, name="CriticAgent", model="gpt-3.5-turbo"):
        super().__init__(name=name, model=model)
        self.system_prompt = (
            "당신은 시를 엄격하게 비판하는 평론가입니다.\n"
            "시를 보고 비판적 코멘트를 남기되, 정말 훌륭하다고 생각하면 반드시 '훌륭한 시야!'라고 말하세요."
        )

# -------------------------------------------------------------------
# 2. GroupChat 구성
# -------------------------------------------------------------------

def run_poetry_creation_with_groupchat(keyword):
    """
    1) UserAgent가 키워드를 입력
    2) PoetAgent가 10줄 시 작성
    3) ComplimentAgent & CriticAgent 각각 코멘트
    4) 만약 둘 다 "훌륭한 시야!"가 나오면 종료, 아니면 PoetAgent가 재창작
    """
    # (1) 에이전트 인스턴스 생성
    user_agent = UserAgent(keyword=keyword)
    poet_agent = PoetAgent()
    compliment_agent = ComplimentAgent()
    critic_agent = CriticAgent()

    # (2) GroupChat 생성
    group_chat = GroupChat(name="PoetryGroupChat")

    # (3) 각 에이전트를 group_chat에 등록
    group_chat.register(user_agent, role="user")            # 실제 사용자 역할
    group_chat.register(poet_agent, role="assistant")       # 시 창작
    group_chat.register(compliment_agent, role="assistant") # 칭찬
    group_chat.register(critic_agent, role="assistant")     # 비판

    print(f"[사용자 키워드] {keyword}\n")

    # (4) 대화 진행을 위한 임시 변수
    all_feedback = ""  # 재창작 시 참고할 피드백들의 누적

    # (5) 루프 돌면서 시 창작 → 칭찬 & 비판 확인 → 재창작
    iteration = 0
    max_iterations = 10  # 무한 루프 방지용 예시

    while iteration < max_iterations:
        iteration += 1
        print(f"=== {iteration}번째 시 창작 ===")

        # 5-1) 먼저 시 창작 에이전트가 발언하도록 GroupChat에 메시지 등록
        # - 사용자(=UserAgent) 메시지: "현재 키워드 & 피드백을 반영해 10줄 시 작성"
        # - 이 메시지에 PoetAgent가 응답하도록 유도
        user_message_for_poet = (
            f"키워드: {keyword}\n"
            f"누적 피드백: {all_feedback}\n"
            "위 정보를 바탕으로 10줄 시를 작성해줘."
        )
        group_chat.post_message(sender=user_agent, message=user_message_for_poet)

        # 5-2) group_chat.step()을 호출하면, `assistant` 역할들의 에이전트가 순서대로 답변 시도
        #   - 현재 등록된 시나리오상, poet_agent가 먼저 나서서 시를 작성할 것으로 기대(기본 순서)
        responses = group_chat.step(max_steps=1)  # 한 번만 돌아보도록
        poem = responses[0].content if responses else ""  # poet_agent가 생성한 시

        print(f"[시 창작 에이전트(PoetAgent)]\n{poem}\n")

        # 5-3) 시에 대한 칭찬/비판 에이전트 각각 코멘트
        # - 마찬가지로 사용자(=UserAgent)가 "이 시를 평가해 달라"고 메시지를 던지고
        #   compliment_agent, critic_agent가 순서대로 답변하도록 유도
        group_chat.post_message(sender=user_agent, message="이 시에 대해 평가해줘.")
        # 두 평론가가 순서대로 답변 (compliment -> critic)
        # 원하는 순서대로 작동하게 하려면 autogen team 기능에서 priority나 name 순서 등에 따라 조정 가능
        # 여기서는 단순히 step 2번 반복
        compliment_response = group_chat.step(max_steps=1)
        critic_response = group_chat.step(max_steps=1)

        compliment_text = compliment_response[0].content if compliment_response else ""
        critic_text = critic_response[0].content if critic_response else ""

        print(f"[칭찬 에이전트(ComplimentAgent)]\n{compliment_text}\n")
        print(f"[비판 에이전트(CriticAgent)]\n{critic_text}\n")

        # 5-4) 종료 조건: 칭찬/비판 모두 "훌륭한 시야!" 문구가 있는지 확인
        if ("훌륭한 시야!" in compliment_text) and ("훌륭한 시야!" in critic_text):
            print("✅ 두 에이전트 모두 시를 만족했습니다. 창작을 종료합니다.")
            break
        else:
            # 만약 만족하지 않으면 -> 피드백 누적 후 다음 루프
            # 시 재창작 시에 참고하기 위해 칭찬/비판 코멘트를 저장
            feedback = (
                f"칭찬 에이전트: {compliment_text}\n"
                f"비판 에이전트: {critic_text}\n"
            )
            all_feedback += feedback + "\n"

        time.sleep(1)

    # (6) 종료
    print("**시 창작 세션이 종료되었습니다.**")


if __name__ == "__main__":
    run_poetry_creation_with_groupchat("봄, 꽃, 바람")
